In [ ]:
# Usage: https://github.com/cwendt94/espn-api/wiki/Football-Intro
from espn_api.football import League, Team, BoxPlayer
from espn_api.football.box_score import BoxScore
from statistics import stdev
import tabulate

TEAM_POSITIONS = ['QB', 'RB', 'RB', 'WR', 'WR', 'TE', 'RB/WR/TE', 'RB/WR/TE', 'K', 'D/ST']

In [ ]:
# private league with cookies
cookie = '<cookie>'
swid = '<swid>'
league_id=0
league = League(league_id=league_id, year=2023, espn_s2=cookie, swid=swid)

In [ ]:
week = 13
box_scores = league.box_scores(week)

In [ ]:
# functions to write
# Put me in, coach (least optimal line up OR highest single bench scorer) OR The Replacements (highest scoring bench)
# Risk it for the biscuit (starting an unlikely starter (<10% start?) and getting rewarded w/ an overperformance)

# Once you have a base-line of 4 weeks:
# "Get dunked on" or "Worldstar-Certified Beatdown" - set the record for worst margin of defeat

# End of Season
# team decimated by injuries? would need to look at questionable, not just IR. Or could look at ADP of drafted players who ended up on IR.
# Draft boon, Draft bust (biggest jump b/w ADP and ROS)
# Waiver winner, Waiver loser (largest contributor off waivers, worst $/point)


# Improvements:
# Dump stats into a spreadsheet. At a minimum, dump team-level stats.
# Use pandas and do JOINS for players, also dump into spreadsheet/update it?
    # If dumping player level information, also record transaction details (e.g., most recent transaction, draft position)

In [ ]:
class Outcome:

    def __init__(self, team: Team, score, projected_score, lineup: list[BoxPlayer], spread):
        self.team = team
        self.score = score
        self.projected_score = projected_score
        self.starters = [p for p in lineup if p.slot_position != "BE"]
        self.starters = sorted(self.starters, key=lambda x: x.points-x.projected_points, reverse=True)
        self.bench = [p for p in lineup if p.slot_position == "BE"]
        self.bench = sorted(self.bench, key=lambda x: x.points-x.projected_points, reverse=True)
        self.spread = spread
        self.won_matchup = spread > 0
        self.boom = self.starters[0]
        self.bust = self.starters[-1]
        self.optimal_lineup = self.get_optimal_lineup()
        self.optimal_score = sum([p.points for p in self.optimal_lineup])
        self.lineup_rating = self.score / self.optimal_score

    def __repr__(self):
        cls = self.__class__.__name__
        return f"{cls}({self.team.owner}, {self.score}, {self.projected_score})"
    
    def get_optimal_lineup(self) -> list[BoxPlayer]:
        players = [*self.starters, *self.bench]
        lineup = []
        for pos in TEAM_POSITIONS:
            eligible_players = [p for p in players if pos in p.eligibleSlots]
            try:
                starter = sorted(eligible_players, key=lambda x: x.points, reverse=True)[0]
                lineup.append(starter)
                players.remove(starter)
            except:
                pass
        
        return lineup
    

def outcomes_from_box_score(box_score: BoxScore) -> list[Outcome]:
    b = box_score
    home_team_spread = box_score.home_score - box_score.away_score
    return [Outcome(b.away_team, b.away_score, b.away_projected, b.away_lineup, -1 * home_team_spread),
            Outcome(b.home_team, b.home_score, b.home_projected, b.home_lineup, home_team_spread)]


In [ ]:
class WeeklyScores:
    
    def __init__(self, _box_scores: list[BoxScore]):
        self.outcomes: list[Outcome] = []
        for box in _box_scores:
            self.outcomes += outcomes_from_box_score(box)
        
        self.outcomes = sorted(self.outcomes, key=lambda o: o.score)
        self._set_all_players()

    def gotta_be_cheating(self) -> bool:
        scores = sorted([o.score for o in self.outcomes])
        top_score = scores[-1]
        next_best = scores[-2]
        return stdev(scores) < top_score - next_best

    def hot_garbage(self) -> bool:
        scores = sorted([o.score for o in self.outcomes])
        low_score = scores[0]
        next_worst = scores[1]
        return stdev(scores) < next_worst - low_score
    
    def format_outcomes(self) -> str:
        table = [[o.team.owner, o.score, o.optimal_score, o.lineup_rating, o.won_matchup, o.spread] for o in self.outcomes]
        return tabulate.tabulate(table,
                                 tablefmt="html",
                                 headers=["Owner", "Score", "OptimalScore", "LineupRating", "Won?", "Spread"])
    
    def format_all_players(self) -> str:
        table = [[p, p.points, p.outcome.team.owner,
                  p.outcome.won_matchup,
                  p.slot_position == "BE"] for p in self.all_players]
        table = sorted(table, key=lambda x: x[1], reverse=True)
        return tabulate.tabulate(table[:50],
                                 tablefmt="html",
                                 headers=["Player", "Points", "Owner", "Won?", "Benched?"],
                                 showindex=True)
    
    def format_busts(self):
        table = [[p, p.points-p.projected_points,
                  p.outcome.team.owner,
                  p.outcome.won_matchup,
                  p.slot_position == "BE"] for p in self.all_players]
        return tabulate.tabulate(table[-1:-16:-1],
                                 tablefmt="html",
                                 headers=["Player", "Bust", "Owner", "Won?", "Benched?"],
                                 showindex=True)
    
    def format_booms(self):
        table = [[p, p.points-p.projected_points,
                  p.outcome.team.owner,
                  p.outcome.won_matchup,
                  p.slot_position == "BE"] for p in self.all_players]
        return tabulate.tabulate(table[:15],
                                 tablefmt="html",
                                 headers=["Player", "Boom", "Owner", "Won?", "Benched?"],
                                 showindex=True)
    
    def _set_all_players(self):
        all_players: list[BoxPlayer] = []
        for outcome in self.outcomes:
            players = [*outcome.starters, *outcome.bench]
            for player in players:
                player.outcome = outcome
                all_players.append(player)
        self.all_players = sorted(all_players,
                                  key=lambda x: x.points - x.projected_points,
                                  reverse=True)
        

weekly_scores = WeeklyScores(box_scores)


In [ ]:
# Section: Overall score awards. Look for hot garbage, gotta be cheating, super salty, busch league,
#                                         big dude, little dude, and rain man

In [ ]:
weekly_scores.hot_garbage(), weekly_scores.gotta_be_cheating()

In [ ]:
weekly_scores.format_outcomes()

In [ ]:
# Section: player awards. Look for Put me in Coach, he's awful my dude, beast mode

In [ ]:
weekly_scores.format_all_players()

In [ ]:
weekly_scores.format_busts()

In [ ]:
weekly_scores.format_booms()